<a href="https://colab.research.google.com/github/AnyaMit/ANLPwTF/blob/main/ANLPwTF_AnyaMit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
%tensorflow_version 2.x
import tensorflow as tf
import os
import io

tf.__version__

'2.5.0'

In [ ]:
# Download the zip file

path_to_zip = tf.keras.utils.get_file("smsspamcollection.zip",origin="https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip",extract=True)

# Unzip the file int a folder

!unzip $path_to_zip -d data

Archive:  /root/.keras/datasets/smsspamcollection.zip
replace data/SMSSpamCollection? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# Let's seee if we  read the data correctly

lines = io.open('data/SMSSpamCollection').read().strip().split('\n')
lines[0]

In [ ]:
## PRE PROCESSING DATA SECTION

In [ ]:
spam_dataset = []
for line in lines:
  label, text = line.split('\t')
  if label.strip() == 'spam':
    spam_dataset.append((1, text.strip()))
  else:
    spam_dataset.append((0, text.strip()))
print(spam_dataset)

In [ ]:
import pandas as pd

df = pd.DataFrame(spam_dataset, columns=['Spam','Message'])

import re

def message_length(x):
  # returns totaal number  of characters
  return len(x)

def num_capitals(x):
  _, count = re.subn(r'[A-Zz]', '', x) # only works in english
  return count

def num_punctuation(x):
  _, count = re.subn(r'\W', '', x) 
  return count




In [ ]:
df['Capitals'] = df['Message'].apply(num_capitals)
df['Punctuation'] = df['Message'].apply(num_punctuation)
df['Length'] = df['Message'].apply(message_length)
df.describe()

In [ ]:
## Split into Test and Train

train = df.sample(frac=0.8, random_state = 42)
test = df.drop(train.index)

x_train = train[['Length','Capitals','Punctuation']]
y_train = train[['Spam']]

x_test = test[['Length','Capitals','Punctuation']]
y_test = test[['Spam']]

In [ ]:
## Model Normalization

# 1- Layer neural network model for evalutaion

def make_model(input_dims = 3, num_units = 12):
  model = tf.keras.Sequential()

  # Add a densely - connected layer with 12 units to the model:
  model.add(tf.keras.layers.Dense(num_units,
                                  input_dim=input_dims,
                                  activation='relu'))
  
  # Add a sigmoid layer with a binary output unit:
  model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

  model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
  
  return model

In [ ]:
model = make_model()

model.fit(x_train, y_train, epochs=10, batch_size=10)

In [ ]:
model.evaluate(x_test,y_test)

y_train_pred = model.predict_classes(x_train)

# confusion matrix

tf.math.confusion_matrix(tf.constant(y_train.Spam),
                         y_train_pred)

#  array([[3733,  134],
#       [ 133,  459]], dtype=int32)>


In [ ]:
sentence = 'Go until Jurong point, crazy.. Available onlly in bugis n great world'
sentence.split()

In [4]:
!pip install stanza 
import stanza


     |████████████████████████████████| 337kB 27.9MB/s 


In [5]:
en = stanza.download('en')

2021-06-18 23:51:10 INFO: Downloading default packages for language: en (English)...


2021-06-18 23:52:32 INFO: Finished downloading models and saved to /root/stanza_resources.


In [7]:
en = stanza.Pipeline(lang='en', processors='tokenize')

2021-06-18 23:55:40 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |

2021-06-18 23:55:40 INFO: Use device: gpu
2021-06-18 23:55:40 INFO: Loading: tokenize
2021-06-18 23:55:40 INFO: Done loading processors!


In [ ]:
tokenized = en(sentence)
len(tokenized.sentences)

for snt in tokenized.sentences:
  for word in snt.tokens:
    print(word.text)
  print("<End of Sentence>")

In [11]:
en = stanza.Pipeline(lang='en') 
print(en)

def word_counts(x, pipeline=en):
  doc = pipeline(x)
  count = sum([len(sentence.tokens) for sentence in doc.sentences])
  return count


2021-06-18 23:55:57 INFO: Done loading processors!


In [12]:
## This did not work due to the error described in the book. Logged an error and got a solution on github -https://github.com/stanfordnlp/stanfordnlp/issues/8

train['Words'] = train['Message'].apply(word_counts)
test['Words'] = test['Message'].apply(word_counts)

x_train = train[['Length', 'Punctuation', 'Capitals', 'Words']]
y_train = train['Spam']

x_test = test[['Length', 'Punctuation', 'Capitals', 'Words']]
y_test = test['Spam']

model = make_model(input_dims=4)

NameError: ignored

In [ ]:
model.fit(x_train,  y_train, epochs=10, batch_size =10)

In [ ]:
train.loc[train.Spam == 1].describe()


In [ ]:
!pip install stopwordsiso

import stopwordsiso as stopwords

stopwords.langs()

In [ ]:
sorted(stopwords.stopwords('en'))

In [ ]:
en_sw = stopwords.stopwords('en')

def word_counts(x, pipeline=en):
  doc = pipeline(x)
  count = 0
  for sentence in doc.sentences:
    for token in sentence.tokens:
      if token.text.lower() not in en_sw:
        count += 1
       # count = count.astype('float32')
  return count

In [ ]:
## Modeling with stopwords removed

train['Words'] = train['Message'].apply(word_counts)
test['Words'] = test['Message'].apply(word_counts)

x_train = train[['Length', 'Punctuation', 'Capitals', 'Words']]
y_train = train['Spam']

x_test = test[['Length', 'Punctuation', 'Capitals', 'Words']]
y_test = test['Spam']

model = make_model(input_dims=4)

/usr/local/lib/python3.7/dist-packages/stanfordnlp/models/depparse/model.py:157: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/cuda/LegacyDefinitions.cpp:28.)
  unlabeled_scores.masked_fill_(diag, -float('inf'))


RuntimeError: ignored

In [ ]:
## POS Tagging

en = snlp.Pipeline(lang='en')
print(en)

txt = "Yo you around? A friend of mine's lookin."
pos = en(txt)

Use device: gpu
---
Loading: tokenize
With settings: 
{'model_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt_tagger.pt', 'pretrain_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt_parser.pt', 'pretrain_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt.pretrain.pt', 'lang': 'en', 'shorthand

RuntimeError: ignored

In [ ]:
X = tfidf.fit_transform(train['Message']).astype('float32')
X_test = tfidf.transform(test['Message']).astype('float32')

X.shape

(4459, 7741)

In [ ]:
def print_pos(doc):
  text = ""
  for sentence in doc.sentences:
    for token in sentence.tokens:
      text += token.words[0].text+"/" + \
              token.words[0].upos+" "
    text += "\n"
  return text

In [ ]:
print(print_pos(pos))

NameError: ignored

In [ ]:
## Skipping page 32-34 as there is an error -- submitted issue https://github.com/stanfordnlp/stanfordnlp/issues/8

In [ ]:
corpus = [
          "I like fruits. Fruits like bananas",
          "I love bananas but eat an apple",
          "An apple a day keeps the doctor away"
]

In [ ]:
!pip install sklearn

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

vectorizer.get_feature_names()

In [ ]:
X.toarray()

array([[0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 2, 0, 0],
       [1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0],
       [1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1]])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(X.toarray())

array([[1.        , 0.13608276, 0.        ],
       [0.13608276, 1.        , 0.3086067 ],
       [0.        , 0.3086067 , 1.        ]])

In [ ]:
query = vectorizer.transform(["apple and bananas"])

cosine_similarity(X, query)

array([[0.23570226],
       [0.57735027],
       [0.26726124]])

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer

transformer = TfidfTransformer(smooth_idf=False)
tfidf = transformer.fit_transform(X.toarray())

pd.DataFrame(tfidf.toarray(),
             columns = vectorizer.get_feature_names())

,an,apple,away,bananas,but,day,doctor,eat,fruits,keeps,like,love,the
0,0.000000,0.000000,0.000000,0.230408,0.000000,0.000000,0.000000,0.000000,0.688081,0.000000,0.688081,0.000000,0.000000
1,0.321267,0.321267,0.000000,0.321267,0.479709,0.000000,0.000000,0.479709,0.000000,0.000000,0.000000,0.479709,0.000000
2,0.275785,0.275785,0.411797,0.000000,0.000000,0.411797,0.411797,0.000000,0.000000,0.411797,0.000000,0.000000,0.411797


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

tfidf = TfidfVectorizer(binary=True)

X = tfidf.fit_transform(train['Message']).astype('float32')
X_test = tfidf.transform(test['Message']).astype('float32')

X.shape

(4459, 7741)

In [ ]:
_, cols = X.shape

model2 = make_model(cols) # to match td-idf dimensions

y_train = train[['Spam']]
y_test = test[['Spam']]

model2.fit(X.toarray(), y_train, epochs = 10, batch_size = 10)

Epoch 1/10
446/446 [==============================] - 1s 3ms/step - loss: 0.3675 - accuracy: 0.8959
Epoch 2/10
446/446 [==============================] - 1s 3ms/step - loss: 0.0991 - accuracy: 0.9782
Epoch 3/10
446/446 [==============================] - 1s 3ms/step - loss: 0.0457 - accuracy: 0.9899
Epoch 4/10
446/446 [==============================] - 1s 3ms/step - loss: 0.0262 - accuracy: 0.9953
Epoch 5/10
446/446 [==============================] - 1s 3ms/step - loss: 0.0161 - accuracy: 0.9973
Epoch 6/10
446/446 [==============================] - 1s 3ms/step - loss: 0.0106 - accuracy: 0.9987
Epoch 7/10
446/446 [==============================] - 1s 3ms/step - loss: 0.0071 - accuracy: 0.9989
Epoch 8/10
446/446 [==============================] - 1s 3ms/step - loss: 0.0050 - accuracy: 0.9993
Epoch 9/10
446/446 [==============================] - 1s 3ms/step - loss: 0.0035 - accuracy: 0.9998
Epoch 10/10
446/446 [==============================] - 1s 3ms/step - loss: 0.0026 - accuracy: 1.0000

In [ ]:
model2.evaluate(X_test.toarray(), y_test)

35/35 [==============================] - 0s 2ms/step - loss: 0.0559 - accuracy: 0.9830


[0.05591704696416855, 0.9829596281051636]

In [ ]:
y_test_pred = model2.predict_classes(X_test.toarray())
tf.math.confusion_matrix(tf.constant(y_test.Spam), y_test_pred)

## Much better performance
## array([[958,   2],
## [ 17, 138]], dtype=int32)>

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[958,   2],
       [ 17, 138]], dtype=int32)>

In [ ]:
!pip install gensim

In [ ]:
from gensim.models.word2vec import Word2Vec
import gensim.downloader as api
model_w2v = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
model_w2v.most_similar("cookies", topn=10)
model_w2v.doesnt_match(["USA","Canada","India","Tokyo"])

king = model_w2v['king']
woman = model_w2v['woman']
man = model_w2v['man']

queen = king - man + woman
model_w2v.similar_by_vector(queen)


NameError: ignored

#### Chapter 2 --- NLU ----

In [ ]:
!pip install tensorflow_datasets
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

In [ ]:
", ".join(tfds.list_builders())

'abstract_reasoning, accentdb, aeslc, aflw2k3d, ag_news_subset, ai2_arc, ai2_arc_with_ir, amazon_us_reviews, anli, arc, bair_robot_pushing_small, bccd, beans, big_patent, bigearthnet, billsum, binarized_mnist, binary_alpha_digits, blimp, bool_q, c4, caltech101, caltech_birds2010, caltech_birds2011, cars196, cassava, cats_vs_dogs, celeb_a, celeb_a_hq, cfq, chexpert, cifar10, cifar100, cifar10_1, cifar10_corrupted, citrus_leaves, cityscapes, civil_comments, clevr, clic, clinc_oos, cmaterdb, cnn_dailymail, coco, coco_captions, coil100, colorectal_histology, colorectal_histology_large, common_voice, coqa, cos_e, cosmos_qa, covid19sum, crema_d, curated_breast_imaging_ddsm, cycle_gan, deep_weeds, definite_pronoun_resolution, dementiabank, diabetic_retinopathy_detection, div2k, dmlab, downsampled_imagenet, dsprites, dtd, duke_ultrasound, emnist, eraser_multi_rc, esnli, eurosat, fashion_mnist, flic, flores, food101, forest_fires, fuss, gap, geirhos_conflict_stimuli, genomics_ood, german_credit

In [ ]:
imdb_train, ds_info = tfds.load(name="imdb_reviews", split="train", with_info = True, as_supervised=True)
imdb_test = tfds.load(name="imdb_reviews",split="test",as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete7BKPYY/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete7BKPYY/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete7BKPYY/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
print(ds_info)

tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
      title     = {Learning Word

In [ ]:
for example, label in imdb_train.take(1):
  print(example, '\n', label)

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string) 
 tf.Tensor(0, shape=(), dtype=int64)


In [ ]:
tokenizer = tfds.deprecated.text.Tokenizer() #Used this instead of what the book used --- why: https://github.com/tensorflow/tensorflow/issues/45217

In [ ]:

vocabulary_set = set()
MAX_TOKENS = 0

for example, label in imdb_train:
  some_tokens = tokenizer.tokenize(example.numpy())
  if MAX_TOKENS < len(some_tokens):
    MAX_TOKENS = len(some_tokens)
  vocabulary_set.update(some_tokens)

In [ ]:
imdb_encoder = tfds.deprecated.text.TokenTextEncoder(vocabulary_set, tokenizer=tokenizer)
vocab_size = imdb_encoder.vocab_size
print(vocab_size, MAX_TOKENS)


93931 2525


In [ ]:
for example, label in imdb_train.take(1):
  print (example)
  encoded = imdb_encoder.encode(example.numpy())
  print(imdb_encoder.decode(encoded))

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string)
This was an absolutely terrible movie Don t be lured in by Christopher Walken or Michael Ironside Both are great actors but this must simply be their worst role in history Even their great acting could not redeem this movie s ridiculous storyline This 

In [ ]:
imdb_encoder.save_to_file("reviews_vocab")
enc = tfds.deprecated.text.TokenTextEncoder.load_from_file("reviews_vocab")
enc.decode(enc.encode("Good case. Excellent value"))

'Good case Excellent value'

In [ ]:
from tensorflow.keras.preprocessing import sequence
def encode_pad_transform(sample):
  encoded = imdb_encoder.encode(sample.numpy())
  pad = sequence.pad_sequences([encoded], padding = 'post', maxlen = 150)
  return np.array(pad[0], dtype=np.int64)

def encode_tf_fn(sample, label):
  encoded = tf.py_function(encode_pad_transform, inp=[sample], Tout = (tf.int64))
  encoded.set_shape([None])
  label.set_shape([])
  return encoded, label

In [ ]:
subset = imdb_train.take(10)
tst = subset.map(encode_tf_fn)
for review, label in tst.take(1):
  print(review,label)
  print(imdb_encoder.decode(review))

tf.Tensor(
[65371 71741 91115 23779 23167 17470 52015 31548 23376 47653 43372 27313
 55821 64804 20625 35429 72538 91546 31601 43203 17503 67540 14579 40436
 45089 23376 24018 18044  3836 43372 49385 90191 24018 43203 87146 88777
 78627 57187 14579 17470 59557 16159 54359 65371 17470 39617 91115  6681
 91632 80751 68350 68445 74217 89911 13960 53891  5565 22925 79914 69807
 58741 43049  5565 20264 24018 35735 26401 60991 86712 81636 75687 16135
 14064 68699 29419 88000  4275 62487 92251 64804 71741 24583 67540 22578
 13960 48683 39475 43372 22578 17470 76500 71741  7679 45005 34840 30157
 30020 71485 49925 36657 76500 60179 31601 57633 31641 14579 73282 14067
 59557 31641 55821 64804 59557 90294 73027 71485 88777  3387 60530 27628
 70193     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0], shape=(150,), dtype=int64) tf.Tensor(0, shape=(), dtype=int64)
This was an

In [ ]:
# Running on the entire set

encoded_train = imdb_train.map(encode_tf_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE)
encoded_test = imdb_test.map(encode_tf_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE)

LSTM model with embeddings

In [ ]:
tf.keras.layers.LSTM(rnn_units) ## another function that doesn't work...

NameError: ignored

In [ ]:
def build_model_lstm(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
          tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                    mask_zero = True,
                                    batch_input_shape =[batch_size, None]),
          tf.keras.layers.LSTM(rnn_units),
          tf.keras.layers.Dense(1,activation='sigmoid')
  ])

  return model

In [ ]:
vocab_size = imdb_encoder.vocab_size

# The embedding dimention
embedding_dim = 64

# Number of RNN units
rnn_units = 64

# batch size
BATCH_SIZE = 100

In [ ]:
model = build_model_lstm(
    vocab_size = vocab_size,
    embedding_dim = embedding_dim,
    rnn_units = rnn_units,
    batch_size = BATCH_SIZE)

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (100, None, 64)           6011584   
_________________________________________________________________
lstm_7 (LSTM)                (100, 64)                 33024     
_________________________________________________________________
dense_5 (Dense)              (100, 1)                  65        
Total params: 6,044,673
Trainable params: 6,044,673
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics = ['accuracy', 'Precision', 'Recall'])

encoded_train_batched = encoded_train.batch(BATCH_SIZE)
model.fit(encoded_train_batched, epochs=10)

Epoch 1/10
250/250 [==============================] - 38s 114ms/step - loss: 0.4106 - accuracy: 0.8044 - precision: 0.7887 - recall: 0.8318
Epoch 2/10
250/250 [==============================] - 28s 113ms/step - loss: 0.1681 - accuracy: 0.9396 - precision: 0.9411 - recall: 0.9379
Epoch 3/10
250/250 [==============================] - 29s 114ms/step - loss: 0.1081 - accuracy: 0.9654 - precision: 0.9663 - recall: 0.9644
Epoch 4/10
250/250 [==============================] - 28s 112ms/step - loss: 0.0831 - accuracy: 0.9714 - precision: 0.9736 - recall: 0.9690
Epoch 5/10
250/250 [==============================] - 28s 113ms/step - loss: 0.0449 - accuracy: 0.9853 - precision: 0.9853 - recall: 0.9854
Epoch 6/10
250/250 [==============================] - 29s 114ms/step - loss: 0.0294 - accuracy: 0.9908 - precision: 0.9911 - recall: 0.9905
Epoch 7/10
250/250 [==============================] - 29s 114ms/step - loss: 0.0314 - accuracy: 0.9901 - precision: 0.9895 - recall: 0.9906
Epoch 8/10
250/250 [

In [ ]:
model.evaluate(encoded_test.batch(BATCH_SIZE))

250/250 [==============================] - 18s 66ms/step - loss: 0.8506 - accuracy: 0.8301 - precision: 0.8363 - recall: 0.8208


[0.8506216406822205, 0.830079972743988, 0.8363221287727356, 0.8208000063896179]

In [ ]:
def build_model_bilstm(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
          tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                    mask_zero = True,
                                    batch_input_shape =[batch_size, None]),
          tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(rnn_units)),
          tf.keras.layers.Dense(1,activation='sigmoid')
  ])

  return model

In [ ]:
bilstm = build_model_bilstm(
    vocab_size = vocab_size,
    embedding_dim = embedding_dim,
    rnn_units = rnn_units,
    batch_size = BATCH_SIZE)

bilstm.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (100, None, 64)           6011584   
_________________________________________________________________
bidirectional_10 (Bidirectio (100, 128)                66048     
_________________________________________________________________
dense_12 (Dense)             (100, 1)                  129       
Total params: 6,077,761
Trainable params: 6,077,761
Non-trainable params: 0
_________________________________________________________________


In [ ]:
bilstm.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics = ['accuracy', 'Precision', 'Recall'])

encoded_train_batched = encoded_train.batch(BATCH_SIZE)

bilstm.fit(encoded_train_batched, epochs=5)

Epoch 1/5
250/250 [==============================] - 36s 120ms/step - loss: 0.4095 - accuracy: 0.8053 - precision: 0.7950 - recall: 0.8227
Epoch 2/5
250/250 [==============================] - 30s 121ms/step - loss: 0.1666 - accuracy: 0.9406 - precision: 0.9414 - recall: 0.9398
Epoch 3/5
250/250 [==============================] - 30s 120ms/step - loss: 0.0832 - accuracy: 0.9738 - precision: 0.9746 - recall: 0.9729
Epoch 4/5
250/250 [==============================] - 30s 121ms/step - loss: 0.0600 - accuracy: 0.9817 - precision: 0.9815 - recall: 0.9820
Epoch 5/5
250/250 [==============================] - 30s 120ms/step - loss: 0.0573 - accuracy: 0.9818 - precision: 0.9815 - recall: 0.9822


Chapter 3: NER

Chapter 4: Transfer Learning with BERT

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd

imdb_train, ds_info = tfds.load(name="imdb_reviews",
                                split="train",
                                with_info=True, as_supervised= True)

imdb_test = tfds.load(name="imdb_reviews", split="test", as_supervised= True)



In [ ]:
# Use default tokenizer settings
tokenizer = tfds.deprecated.text.Tokenizer()

vocabulary_set = set()
MAX_TOKENS = 0

for example, label in imdb_train:
  some_tokens = tokenizer.tokenize(example.numpy())
  if MAX_TOKENS < len(some_tokens):
    MAX_TOKENS = len(some_tokens)
  vocabulary_set.update(some_tokens)


In [ ]:
imdb_encoder = tfds.deprecated.text.TokenTextEncoder(vocabulary_set, lowercase = True, tokenizer=tokenizer)
vocab_size = imdb_encoder.vocab_size
print(vocab_size, MAX_TOKENS)

93931 2525


In [ ]:
from tensorflow.keras.preprocessing import sequence

def encode_pad_transform(sample):
  encoded = imdb_encoder.encode(sample.numpy())
  pad = sequence.pad_sequences([encoded], padding = 'post', maxlen = 150)
  return np.array(pad[0], dtype=np.int64)

def encode_tf_fn(sample, label):
  encoded = tf.py_function(encode_pad_transform, inp=[sample], Tout = (tf.int64))
  encoded.set_shape([None])
  label.set_shape([])
  return encoded, label

In [ ]:
# Running on the entire set

encoded_train = imdb_train.map(encode_tf_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE)
encoded_test = imdb_test.map(encode_tf_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
## Download pre-trained embeddings
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

glove.6B.zip        100%[===================>] 822.24M  5.00MB/s    in 2m 43s  

2021-06-18 20:34:28 (5.05 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:
dict_w2v = {}
with open('glove.6B.50d.txt',"r") as file:
  for line in file:
    tokens = line.split()
    word = tokens[0]
    vector = np.array(tokens[1:], dtype=np.float32)

    if vector.shape[0] == 50:
      dict_w2v[word] = vector
    else:
      print("There was an issue with " + word)

# Lets check the vocab size
print("Dictionary Size:", len(dict_w2v))

Dictionary Size: 400000


In [ ]:
embedding_dim = 50
embedding_matrix = np.zeros((imdb_encoder.vocab_size, embedding_dim))

In [ ]:
unk_cnt = 0
unk_set = set()
for word in imdb_encoder.tokens:
  embedding_vector = dict_w2v.get(word)

  if embedding_vector is not None:
    tkn_id = imdb_encoder.encode(word)[0]
    embedding_matrix[tkn_id] = embedding_vector
  else:
    unk_cnt += 1
    unk_set.add(word)

# Print how many weren't found
print("Total unknown words:", unk_cnt)

Total unknown words: 14553


In [ ]:
##################### FEATURE EXTRACTION MODEL ##########################

In [ ]:
# Length of vocab in chars
vocab_size = imdb_encoder.vocab_size #len chars

# Number of RNN units
rnn_units = 64

# batch size
BATCH_SIZE = 100

In [ ]:
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense

def build_model_bilstm(vocab_size, embedding_dim, rnn_units, batch_size, train_emb=False):
  model = tf.keras.Sequential([
          Embedding(vocab_size, embedding_dim,
                                    mask_zero = True,
                                    weights = [embedding_matrix],
                                    trainable = train_emb),
  Bidirectional(LSTM(rnn_units, return_sequences=True,dropout=0.5)),
  Bidirectional(LSTM(rnn_units,dropout=0.25)),
  Dense(1, activation='sigmoid')
  ])

  return model

In [ ]:
model_fe = build_model_bilstm(
    vocab_size = vocab_size,
    embedding_dim = embedding_dim,
    rnn_units = rnn_units,
    batch_size = BATCH_SIZE)

model_fe.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, None, 50)          4696550   
_________________________________________________________________
bidirectional_11 (Bidirectio (None, None, 128)         58880     
_________________________________________________________________
bidirectional_12 (Bidirectio (None, 128)               98816     
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 129       
Total params: 4,854,375
Trainable params: 157,825
Non-trainable params: 4,696,550
_________________________________________________________________


In [ ]:
model_fe.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics = ['accuracy', 'Precision', 'Recall'])

encoded_train_batched = encoded_train.batch(BATCH_SIZE).prefetch(100)

model_fe.fit(encoded_train_batched, epochs=10)

Epoch 1/10
250/250 [==============================] - 33s 79ms/step - loss: 0.5959 - accuracy: 0.6763 - precision: 0.6849 - recall: 0.6530
Epoch 2/10
250/250 [==============================] - 21s 85ms/step - loss: 0.5259 - accuracy: 0.7387 - precision: 0.7433 - recall: 0.7294
Epoch 3/10
250/250 [==============================] - 21s 84ms/step - loss: 0.4936 - accuracy: 0.7624 - precision: 0.7692 - recall: 0.7496
Epoch 4/10
250/250 [==============================] - 21s 86ms/step - loss: 0.4731 - accuracy: 0.7748 - precision: 0.7803 - recall: 0.7650
Epoch 5/10
250/250 [==============================] - 22s 87ms/step - loss: 0.4599 - accuracy: 0.7800 - precision: 0.7809 - recall: 0.7786
Epoch 6/10
250/250 [==============================] - 21s 85ms/step - loss: 0.4391 - accuracy: 0.7957 - precision: 0.7956 - recall: 0.7959
Epoch 7/10
250/250 [==============================] - 21s 86ms/step - loss: 0.4253 - accuracy: 0.8053 - precision: 0.8047 - recall: 0.8062
Epoch 8/10
250/250 [=======

In [ ]:
model_fe.evaluate(encoded_test.batch(BATCH_SIZE))

250/250 [==============================] - 22s 70ms/step - loss: 0.3934 - accuracy: 0.8378 - precision: 0.8062 - recall: 0.8894


[0.39340922236442566,
 0.8377599716186523,
 0.8061639070510864,
 0.8893600106239319]

In [ ]:
##################### FINE TUNING MODEL ##########################

In [ ]:
model_ft = build_model_bilstm(
    vocab_size = vocab_size,
    embedding_dim = embedding_dim,
    rnn_units = rnn_units,
    batch_size = BATCH_SIZE,
    train_emb = True)

model_ft.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, None, 50)          4696550   
_________________________________________________________________
bidirectional_13 (Bidirectio (None, None, 128)         58880     
_________________________________________________________________
bidirectional_14 (Bidirectio (None, 128)               98816     
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 129       
Total params: 4,854,375
Trainable params: 4,854,375
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_ft.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics = ['accuracy', 'Precision', 'Recall'])

encoded_train_batched = encoded_train.batch(BATCH_SIZE).prefetch(100)

model_ft.fit(encoded_train_batched, epochs=10)

Epoch 1/10
250/250 [==============================] - 44s 124ms/step - loss: 0.5560 - accuracy: 0.7066 - precision: 0.7067 - recall: 0.7063
Epoch 2/10
250/250 [==============================] - 32s 128ms/step - loss: 0.4015 - accuracy: 0.8207 - precision: 0.8248 - recall: 0.8144
Epoch 3/10
250/250 [==============================] - 32s 128ms/step - loss: 0.3297 - accuracy: 0.8601 - precision: 0.8625 - recall: 0.8567
Epoch 4/10
250/250 [==============================] - 32s 128ms/step - loss: 0.2780 - accuracy: 0.8870 - precision: 0.8904 - recall: 0.8828
Epoch 5/10
250/250 [==============================] - 32s 127ms/step - loss: 0.2441 - accuracy: 0.8995 - precision: 0.9027 - recall: 0.8956
Epoch 6/10
250/250 [==============================] - 32s 127ms/step - loss: 0.2097 - accuracy: 0.9190 - precision: 0.9213 - recall: 0.9163
Epoch 7/10
250/250 [==============================] - 32s 128ms/step - loss: 0.1844 - accuracy: 0.9299 - precision: 0.9316 - recall: 0.9279
Epoch 8/10
250/250 [

In [ ]:
!pip install transformers==3.0.2

     |████████████████████████████████| 778kB 7.1MB/s 
     |████████████████████████████████| 3.0MB 13.4MB/s 
     |████████████████████████████████| 901kB 34.5MB/s 
     |████████████████████████████████| 1.2MB 44.5MB/s 


In [ ]:
from transformers import BertTokenizer
bert_name = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(bert_name,
                                          add_special_tokens=True,
                                          do_lower_case = False,
                                          max_length=150,
                                          pad_to_max_length=True)

In [ ]:
tokenizer.encode_plus("Don't be lured", add_special_tokens=True,
                      max_length = 9,
                      pad_to_max_length = True,
                      return_attention_mask = True,
                      return_token_type_ids=True)

{'input_ids': [101, 1790, 112, 189, 1129, 19615, 1181, 102, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 0]}

In [ ]:
tokenizer.encode_plus(" Don't be", " lured", add_special_tokens=True,
                      max_length = 10,
                      pad_to_max_length = True,
                      return_attention_mask = True,
                      return_token_type_ids=True)

{'input_ids': [101, 1790, 112, 189, 1129, 102, 19615, 1181, 102, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 1, 1, 1, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 0]}

In [ ]:
def bert_encoder(review):
  txt = review.numpy().decode('utf-8')
  encoded = tokenizer.encode_plus(txt, add_special_tokens=True,
                                  max_length = 150,
                                  pad_to_max_length = True,
                                  return_attention_mask = True,
                                  return_token_type_ids=True)
  
  return encoded['input_ids'], encoded['token_type_ids'], encoded['attention_mask']

In [ ]:
bert_train = [bert_encoder(r) for r, l in imdb_train]
bert_lbl = [l for r, l in imbd_train]
bert_train = np.array(bert_train)
bert_lbl = tf.keras.utils.to_categorical(bert_lbl, num_classes=2)

Streaming output truncated to the last 5000 lines.


NameError: ignored

In [ ]:
# create training and validation splits 
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_Split(bert_train,
bert_lbl,
test_size=0.2,
random_state=42))